# 简介
在这个教程中，我们将介绍pyecharts包。  
  
Echarts是百度开发的一个javascript可视化脚本，他的特点在于可交互（就是你鼠标移动上去，图片可以作出相应的变化）。

echarts属于门槛比较高的一种可视化方法，需要写javascript，而javascript又是基于html网页，另外数据的处理还要用python，等于说你要熟练使用得同时会js，html，python三种语言。现在pyecharts出现了，直接在python里就可以调用生成echarts图片，大部分的图表只需要用python就可以生成，非常方便！  
  
  
>matplotlib：纯python出图，可批量出图，缺点是出的图片为静态图片无法交互  
folium：主要功能是绘制地图，javascript出图可交互，坐标系为wgs84，数据不需要转坐标  
echarts：可绘制各种图表，也能绘制地图，javascript出图可交互，但绘制地图时底图一般采用百度地图，需要转坐标系



# 数据获取

很多网站都提供了疫情分布情况，数据都是公开的我们直接抓就行。这里以腾讯新闻的疫情发布链接为例，观察网络链接可以找到数据获取的访问请求

<img src="https://gitee.com/ni1o1/pygeo-tutorial/raw/master/resource/yiqing.png"  style="width:800px">  

好的我们用最简单的方式把它抓下来，我们做省份的可视化

In [1]:
import urllib
import json
url = 'https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5'
request = urllib.request.Request(url)
response = urllib.request.urlopen(request)
datajson=json.loads(response.read().decode('utf8'))
datajson=json.loads(datajson['data'])


In [2]:
#数据就存放在这个变量里
datajson

{'lastUpdateTime': '2022-02-21 14:14:42',
 'chinaTotal': {'confirm': 150454,
  'heal': 131502,
  'dead': 5776,
  'nowConfirm': 13176,
  'suspect': 3,
  'nowSevere': 10,
  'importedCase': 13472,
  'noInfect': 686,
  'showLocalConfirm': 1,
  'showlocalinfeciton': 1,
  'localConfirm': 841,
  'noInfectH5': 114,
  'localConfirmH5': 841,
  'local_acc_confirm': 107851},
 'chinaAdd': {'confirm': 603,
  'heal': 269,
  'dead': 12,
  'nowConfirm': 322,
  'suspect': 3,
  'nowSevere': 3,
  'importedCase': 73,
  'noInfect': 38,
  'localConfirm': 19,
  'noInfectH5': 15,
  'localConfirmH5': 71},
 'isShowAdd': True,
 'showAddSwitch': {'all': True,
  'confirm': True,
  'suspect': True,
  'dead': True,
  'heal': True,
  'nowConfirm': True,
  'nowSevere': True,
  'importedCase': True,
  'noInfect': True,
  'localConfirm': True,
  'localinfeciton': True},
 'areaTree': [{'name': '中国',
   'today': {'confirm': 603, 'isUpdated': True},
   'total': {'nowConfirm': 13176,
    'confirm': 150454,
    'dead': 5776,


In [3]:
#提取各省份的数据
import pandas as pd
provincedata = pd.DataFrame(datajson['areaTree'][0]['children'])
provincedata.head(5)

,name,today,total,children
0,香港,"{'confirm': 340, 'confirmCuts': 0, 'isUpdated'...","{'nowConfirm': 5990, 'confirm': 22468, 'dead':...","[{'name': '地区待确认', 'today': {'confirm': 340, '..."
1,台湾,"{'confirm': 119, 'confirmCuts': 0, 'isUpdated'...","{'nowConfirm': 5462, 'confirm': 20056, 'dead':...","[{'name': '地区待确认', 'today': {'confirm': 119, '..."
2,浙江,"{'confirm': 0, 'confirmCuts': 0, 'isUpdated': ...","{'nowConfirm': 383, 'confirm': 2255, 'dead': 1...","[{'name': '杭州', 'today': {'confirm': 0, 'confi..."
3,广东,"{'confirm': 26, 'confirmCuts': 0, 'isUpdated':...","{'nowConfirm': 362, 'confirm': 4163, 'dead': 8...","[{'name': '境外输入', 'today': {'confirm': 20, 'co..."
4,广西,"{'confirm': 5, 'confirmCuts': 0, 'isUpdated': ...","{'nowConfirm': 319, 'confirm': 1028, 'dead': 2...","[{'name': '百色', 'today': {'confirm': 0, 'confi..."


In [4]:
#整理一下数据，把total里面的数据展开
data1 = pd.DataFrame(list(provincedata['total']))
data1['name'] = provincedata['name']
data1.head(5)

,nowConfirm,confirm,dead,showRate,heal,showHeal,wzz,provinceLocalConfirm,name
0,5990,22468,288,False,16190,True,0,0,香港
1,5462,20056,852,False,13742,True,0,0,台湾
2,383,2255,1,False,1871,True,7,1868,浙江
3,362,4163,8,False,3793,True,14,1720,广东
4,319,1028,2,False,707,True,0,546,广西


好的，到这一步我们已经获取了数据

# 可视化

## 全国数据可视化

官方配置文档：[pyecharts的地理图表教程](https://pyecharts.org/#/zh-cn/geography_charts)  
首先我们要把数据整理成echarts认识的格式，就是如下：

In [4]:
data1[['name','confirm']].values

array([['湖北', 16678],
       ['浙江', 895],
       ['广东', 870],
       ['河南', 764],
       ['湖南', 661],
       ['江西', 548],
       ['安徽', 530],
       ['重庆', 366],
       ['江苏', 341],
       ['四川', 301],
       ['山东', 298],
       ['北京', 253],
       ['上海', 233],
       ['福建', 205],
       ['黑龙江', 190],
       ['陕西', 165],
       ['广西', 150],
       ['河北', 135],
       ['云南', 122],
       ['海南', 91],
       ['辽宁', 81],
       ['山西', 81],
       ['天津', 69],
       ['贵州', 64],
       ['甘肃', 57],
       ['吉林', 54],
       ['内蒙古', 42],
       ['宁夏', 34],
       ['新疆', 32],
       ['香港', 18],
       ['青海', 17],
       ['台湾', 11],
       ['澳门', 10],
       ['西藏', 1]], dtype=object)

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Map
#创建echarts对象c
c = (
    Map()#告诉echarts这个是Map形式的图表
    .add("确诊", data1[['name','confirm']].values, "china")#加一个数据，这个数据名叫”确诊“，数据的地图是echarts自带的china
    .set_global_opts(#对图表添加设置
        title_opts=opts.TitleOpts(title='疫情地图')#设置图表的名称
    )
)
#导出为html文件
c.render('疫情地图.html')

打开目录下的 [疫情地图.html](疫情地图.html) 文件，效果如下
<img src="https://gitee.com/ni1o1/pygeo-tutorial/raw/master/resource/地图v1.png"  style="width:500px">  
但是，我们还要在加一些美化调整，一些参数可以看[echarts官方的配置项手册](https://www.echartsjs.com/zh/option.html#title)

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Map
c = (
    Map()
    .add("确诊", 
         data1[['name','confirm']].values, 
         "china",
         is_roam = False,#不可鼠标缩放和平移漫游
         zoom = 1.2,#当前视角的缩放比例
         is_map_symbol_show = False, # 是否显示标记图形
         label_opts = opts.LabelOpts(position = 'inside'),#标签尽量放在图形区域内
        )
    .set_global_opts(
        title_opts=opts.TitleOpts(title='疫情地图'),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True,#设定分段颜色显示
                                          pieces=[{'min': 10000,'label':'10000人以上'}, #设定分段的值
                                                  {'min': 1000, 'max': 9999,'label':'1000-9999人'},
                                                  {'min': 500, 'max': 999,'label':'500-999人'},
                                                  {'min': 100, 'max': 499,'label':'100-499人'},
                                                  {'min': 10, 'max': 99,'label':'10-99人'},
                                                  {'min': 1, 'max': 9,'label':'1-9人'}],
                                           range_color=["#b4e0f3","#70b4eb","#1482e5","#1c3fbf","#070093" ] #调整显示颜色
                                         ),
    )
)
c.render('疫情地图.html')


打开目录下的 [疫情地图.html](疫情地图.html) 文件，效果如下
<img src="https://gitee.com/ni1o1/pygeo-tutorial/raw/master/resource/地图2.png"  style="width:500px">  

## 单个省份数据可视化

In [50]:
import pandas as pd
#提取省份的数据
province = '广东'
guangdongdata = pd.DataFrame(provincedata[provincedata['name'] == province]['children'].iloc[0])
#整理一下数据，把total里面的数据展开
data2 = pd.DataFrame(list(guangdongdata['total']))
data2['name'] = guangdongdata['name']+'市'
data2.head(5)

,confirm,suspect,dead,heal,name
0,289,0,0,13,深圳市
1,237,0,0,3,广州市
2,69,0,0,1,珠海市
3,49,0,0,1,佛山市
4,44,0,0,0,东莞市


In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Map
c = (
    Map()
    .add("确诊", 
         data2[['name','confirm']].values, 
         province,
         is_roam = False,#不可鼠标缩放和平移漫游
         zoom = 1.2,#当前视角的缩放比例
         is_map_symbol_show = False, # 是否显示标记图形
         label_opts = opts.LabelOpts(position = 'inside'),#标签尽量放在图形区域内
        )
    .set_global_opts(
        title_opts=opts.TitleOpts(title=province+'疫情地图'),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True,#设定分段颜色显示
                                          pieces=[{'min': 200, 'label':'200人以上'},#设定分段的值
                                                  {'min': 100, 'max': 199,'label':'100-199人'},
                                                  {'min': 50, 'max': 99,'label':'50-99人'},
                                                  {'min': 10, 'max': 49,'label':'10-49人'},
                                                  {'min': 1, 'max': 9,'label':'1-9人'}],
                                           range_color=["#b4e0f3","#70b4eb","#1482e5","#1c3fbf","#070093" ] #调整显示颜色
                                         ),
    )
)
c.render(province+'疫情地图.html')


打开目录下的 [广东疫情地图.html](广东疫情地图.html) 文件，效果如下
<img src="https://gitee.com/ni1o1/pygeo-tutorial/raw/master/resource/广东疫情地图.png"  style="width:500px">  